In [4]:
import onnxruntime as ort
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoImageProcessor
from PIL import Image
import requests

# Load the image processor using the preprocessor config
processor = AutoImageProcessor.from_pretrained("/workspace/onnx-nomic-embed-v1.5")

# Load the ONNX model
onnx_model_path = "/workspace/onnx-nomic-embed-v1.5/model.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["TensorrtExecutionProvider", "CPUExecutionProvider"])

# Load an image
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

# Preprocess the image
inputs = processor(image, return_tensors="np")
onnx_inputs = {session.get_inputs()[0].name: inputs["pixel_values"]}

# Run inference
onnx_outputs = session.run(None, onnx_inputs)

# Extract and normalize embeddings
img_emb = torch.tensor(onnx_outputs[0])
img_embeddings = F.normalize(img_emb[:, 0], p=2, dim=1)

print("Image Embeddings:", img_embeddings.shape)


*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:507 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************


2025-02-27 23:48:24.269165215 [E:onnxruntime:Default, provider_bridge_ort.cc:1848 TryGetProviderInfo_TensorRT] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1539 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libnvinfer.so.10: cannot open shared object file: No such file or directory



Image Embeddings: torch.Size([1, 768])


In [2]:
import onnxruntime as ort
print(ort.get_available_providers())


['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [9]:
img_embeddings

tensor([[ 4.7530e-03, -2.5440e-02,  6.6818e-03, -2.9590e-02, -4.3530e-02,
         -1.2267e-02,  2.4384e-03, -3.6055e-02, -7.0345e-02, -1.8580e-02,
         -4.4000e-02, -1.7478e-02, -1.3518e-02, -6.2597e-02, -6.4252e-02,
         -2.5552e-02,  5.2608e-03, -4.0555e-02, -1.8450e-02,  1.6208e-03,
         -5.7390e-02, -1.1387e-02, -2.7568e-02, -1.4647e-02,  3.7186e-02,
         -5.9439e-02, -4.8858e-02, -2.7063e-02, -4.7970e-02, -2.9087e-02,
         -4.3004e-02, -5.3142e-02, -1.6650e-02, -2.7782e-02, -3.5605e-02,
          1.0692e-02, -3.6191e-02, -2.5946e-02, -6.0531e-02, -2.0661e-02,
         -3.4341e-02, -3.1145e-02, -3.5184e-02,  1.1067e-02, -4.6758e-02,
          2.6561e-03, -2.0196e-02, -5.6646e-02, -4.4105e-02, -3.9693e-02,
         -3.0172e-02, -4.7597e-02, -1.8874e-02, -1.5512e-02, -5.8379e-02,
         -1.3859e-02, -9.0496e-02, -2.7282e-02, -2.0452e-02,  8.5287e-03,
         -2.0871e-02, -3.2452e-02, -2.3347e-02, -1.6193e-02, -4.2960e-02,
          3.5033e-02, -1.2709e-02, -6.

In [6]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForImageClassification

model_checkpoint = "nomic-ai/nomic-embed-vision-v1.5"
save_directory = "onnx-nomic-embed-vision-v1.5/"

In [ ]:
# Load a model from transformers and export it to ONNX
ort_model = ORTModelForImageClassification.from_pretrained(model_checkpoint, export=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
# Load ONNX - get embeddings

In [59]:
import pandas as pd
df_products = pd.read_csv("final_products.csv")
result_array = df_products.apply(lambda row: f"Title of Product: {row['title']}\nProduct Image Description: {row['llava_generated_image_caption']}\nProduct Category: {row['category_name']}", axis=1).tolist()
result_array[:2]

['Title of Product: Original Replacement Dell 130W Laptop Charger USB C Slim AC Power Adapter for Dell Xps 17,Precision 5550 5530 2in1,XPS 15 2in1 9575，DA130PM170 HA130PM170 0K00F5 K00F5 0M0H25 M0H25 T4V18\nProduct Image Description: A black power bank, which is a portable charger used to charge electronic devices.\nProduct Category: Laptop Accessories',
 'Title of Product: Griffin Elevator Stand for Laptops - Lift Your Laptop to a Comfortable Viewing Height, Space Grey\nProduct Image Description: A laptop computer sitting on a stand or a docking station.\nProduct Category: Laptop Accessories']

In [61]:
from transformers import AutoTokenizer
from onnxruntime import InferenceSession



tokenizer = AutoTokenizer.from_pretrained("WhereIsAI/UAE-Large-V1")

outputs = []
inputs = [
    'Title of Product: Original Replacement Dell 130W Laptop Charger USB C Slim AC Power Adapter for Dell Xps 17,Precision 5550 5530 2in1,XPS 15 2in1 9575，DA130PM170 HA130PM170 0K00F5 K00F5 0M0H25 M0H25 T4V18\nProduct Image Description: A black power bank, which is a portable charger used to charge electronic devices.\nProduct Category: Laptop Accessories',
    'Title of Product: Griffin Elevator Stand for Laptops - Lift Your Laptop to a Comfortable Viewing Height, Space Grey\nProduct Image Description: A laptop computer sitting on a stand or a docking station.\nProduct Category: Laptop Accessories'
]

session = InferenceSession("onnx/model.onnx")

query_string = tokenizer(str("a black thing"), return_tensors="np")
query_embedding = session.run(output_names=["last_hidden_state"], input_feed=dict(query_string))

for item in inputs:
    input_tokens = tokenizer(str(item), return_tensors="np")
    # print(item)
    outputs.append(session.run(output_names=["last_hidden_state"], input_feed=dict(input_tokens)))



In [62]:
item_number = 0
outputs[item_number][0][0][0]
# [-0.39422712, -0.71218926,  0.25285605, ..., -0.55258805,
#         -0.31103668, -1.112754  ]

array([-0.6131097 , -0.90806764,  0.36052167, ..., -0.17835039,
        0.14225149, -0.47162902], shape=(1024,), dtype=float32)

In [63]:
query_embedding[0][0][0]

array([-0.34115973, -0.7469413 , -0.06696676, ...,  0.06408735,
       -0.01018069,  0.11800822], shape=(1024,), dtype=float32)

In [64]:
from scipy import spatial

for dv in outputs:
    print(1 - spatial.distance.cosine(query_embedding[0][0][0], dv[0][0][0]))
    # print(dv[0][0][0])

0.5139134
0.5181762


In [42]:
from transformers.models.distilbert import DistilBertConfig, DistilBertOnnxConfig

config = DistilBertConfig()
onnx_config = DistilBertOnnxConfig(config)
print(list(onnx_config.outputs.keys()))

['last_hidden_state']


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("distilbert_base_uncased_squad_onnx", backend="onnx")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: distilbert_base_uncased_squad_onnx
2025-02-27 12:41:23.205791088 [W:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2025-02-27 12:41:23 WARNING] ModelImporter.cpp:459: Make sure input input_ids has Int64 binding.
2025-02-27 12:41:23.205836858 [W:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2025-02-27 12:41:23 WARNING] ModelImporter.cpp:459: Make sure input attention_mask has Int64 binding.
2025-02-27 12:41:24.998398544 [W:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2025-02-27 12:41:24 WARNING] ModelImporter.cpp:459: Make sure input input_ids has Int64 binding.
2025-02-27 12:41:24.998429764 [W:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2025-02-27 12:41:24 WARNING] ModelImporter.cpp:459: Make sure input attention_mask has Int64 binding.
Batches: 100%|██████████| 1/1 [00:

In [31]:
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)
embeddings

Batches: 100%|██████████| 1/1 [00:00<00:00, 240.87it/s]


array([[ 0.14351264, -0.13583307,  0.648162  , ..., -0.08970557,
         0.28320345, -0.19161642],
       [ 0.12950341, -0.48151484,  0.23586589, ..., -0.42174533,
        -0.2797937 , -0.28718296]], shape=(2, 1024), dtype=float32)

In [17]:
embeddings

array([[ 0.14351264, -0.13583307,  0.648162  , ..., -0.08970557,
         0.28320345, -0.19161642],
       [ 0.12950341, -0.48151484,  0.23586589, ..., -0.42174533,
        -0.2797937 , -0.28718296]], shape=(2, 1024), dtype=float32)